In [1]:
import os
import torch

# Setting up the environment to use only GPU 2
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check number of GPUs
print(f"Number of GPUs: {torch.cuda.device_count()}")

# Check GPU name
print(f"GPU name: {torch.cuda.get_device_name()}")

# Check PyTorch version
print(f"PyTorch version: {torch.__version__}")

Using device: cuda
Number of GPUs: 1
GPU name: NVIDIA A100-SXM4-40GB
PyTorch version: 2.5.1+cu121


In [2]:
!nvidia-smi

Tue Dec  3 19:32:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   35C    P0             240W / 400W |  12502MiB / 40960MiB |     99%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [7]:
from PIL import Image
import json
import os

# Define the directory containing the images and annotations
image_directory = "/data/students/mary/mlops-exercises/ME6/dataset__all/train/images"
annotation_file = "/data/students/mary/mlops-exercises/ME6/dataset__v3/instances_train.json"

# Load the COCO annotations
with open(annotation_file, 'r') as f:
    coco_data = json.load(f)

# Iterate through each image file in the directory
for filename in os.listdir(image_directory):
    if filename.startswith("09") and filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        # Extract the numeric part of the filename and compare
        file_number = int(filename.split('.')[0])
        if file_number > 90350:
            # Open the image
            image_path = os.path.join(image_directory, filename)
            with Image.open(image_path) as img:
                width, height = img.size
                
                # Check if the image is in portrait orientation
                if height > width:

                    # Interchange the width and height of the images in the training set in the json file
                    # Adjust the image dimensions
                    for img_info in coco_data['images']:
                        if img_info['file_name'] == filename:
                            img_info['width'] = width
                            img_info['height'] = height

                    # Find the image ID in the COCO annotations
                    img_id = None
                    for img_info in coco_data['images']:
                        if img_info['file_name'] == filename:
                            img_id = img_info['id']
                            break
                    
                    if img_id:
                        # Adjust the annotations for the image
                        for ann in coco_data['annotations']:
                            if ann['image_id'] == img_id:
                                # Adjust bounding box
                                x, y, w, h = ann['bbox']
                                if y > width/2:
                                    dx = y - width/2
                                    y = round(y - 2*dx - h,1)
                                else:
                                    dx = width/2 - y
                                    y = round(y + 2*dx - h, 1)

                                ann['bbox'] = [y, x, h, w]
                                
                                # Adjust segmentation points
                                if 'segmentation' in ann:
                                    new_segmentation = []
                                    for seg in ann['segmentation']:
                                        new_seg = []
                                        for i in range(0, len(seg), 2):
                                            if seg[i+1] > width/2:
                                                dx = seg[i+1] - width/2
                                                new_seg.append(round(seg[i+1] - 2*dx,1))
                                            else:
                                                dx = width/2 - seg[i+1]
                                                new_seg.append(round(seg[i+1] + 2*dx,1))
                                            new_seg.append(seg[i])

                                        new_segmentation.append(new_seg)
                                    ann['segmentation'] = new_segmentation

# Save the adjusted annotations
adj_filename = f"/data/students/mary/mlops-exercises/ME6/dataset__v3/adj_instances_train.json"
with open(adj_filename, 'w') as f:
    json.dump(coco_data, f)

In [8]:
from PIL import Image
import json
import os

# Define the directory containing the images and annotations
image_directory = "/data/students/mary/mlops-exercises/ME6/dataset__all/val/images"
annotation_file = "/data/students/mary/mlops-exercises/ME6/dataset__v3/instances_val.json"

# Load the COCO annotations
with open(annotation_file, 'r') as f:
    coco_data = json.load(f)

# Iterate through each image file in the directory
for filename in os.listdir(image_directory):
    if filename.startswith("09") and filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        # Extract the numeric part of the filename and compare
        file_number = int(filename.split('.')[0])
        if file_number > 90350:
            # Open the image
            image_path = os.path.join(image_directory, filename)
            with Image.open(image_path) as img:
                width, height = img.size
                
                # Check if the image is in portrait orientation
                if height > width:

                    # Interchange the width and height of the images in the training set in the json file
                    # Adjust the image dimensions
                    for img_info in coco_data['images']:
                        if img_info['file_name'] == filename:
                            img_info['width'] = width
                            img_info['height'] = height

                    # Find the image ID in the COCO annotations
                    img_id = None
                    for img_info in coco_data['images']:
                        if img_info['file_name'] == filename:
                            img_id = img_info['id']
                            break
                    
                    if img_id:
                        # Adjust the annotations for the image
                        for ann in coco_data['annotations']:
                            if ann['image_id'] == img_id:
                                # Adjust bounding box
                                x, y, w, h = ann['bbox']
                                if y > width/2:
                                    dx = y - width/2
                                    y = round(y - 2*dx - h,1)
                                else:
                                    dx = width/2 - y
                                    y = round(y + 2*dx - h, 1)

                                ann['bbox'] = [y, x, h, w]
                                
                                # Adjust segmentation points
                                if 'segmentation' in ann:
                                    new_segmentation = []
                                    for seg in ann['segmentation']:
                                        new_seg = []
                                        for i in range(0, len(seg), 2):
                                            if seg[i+1] > width/2:
                                                dx = seg[i+1] - width/2
                                                new_seg.append(round(seg[i+1] - 2*dx,1))
                                            else:
                                                dx = width/2 - seg[i+1]
                                                new_seg.append(round(seg[i+1] + 2*dx,1))
                                            new_seg.append(seg[i])

                                        new_segmentation.append(new_seg)
                                    ann['segmentation'] = new_segmentation

# Save the adjusted annotations
adj_filename = f"/data/students/mary/mlops-exercises/ME6/dataset__v3/adj_instances_val.json"
with open(adj_filename, 'w') as f:
    json.dump(coco_data, f)